In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("abc").master("spark://spark-master:7077").getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.types import StructType,StructField, StringType, DoubleType

schema = StructType([
      StructField("sbd",StringType(),True),
      StructField("toan",DoubleType(),True),
      StructField("ngu_van",DoubleType(),True),
      StructField("ngoai_ngu",DoubleType(),True),
      StructField("vat_li",DoubleType(),True),
      StructField("hoa_hoc",DoubleType(),True),
      StructField("sinh_hoc",DoubleType(),True),
      StructField("lich_su",DoubleType(),True),
      StructField("dia_li",DoubleType(),True),
      StructField("gdcd",DoubleType(),True),
  ])

In [5]:
data_frame = spark.read.schema(schema) \
    .csv("hdfs://namenode:9000/data/*.csv", header=True)
data_frame.printSchema()
data_frame.show()

root
 |-- sbd: string (nullable = true)
 |-- toan: double (nullable = true)
 |-- ngu_van: double (nullable = true)
 |-- ngoai_ngu: double (nullable = true)
 |-- vat_li: double (nullable = true)
 |-- hoa_hoc: double (nullable = true)
 |-- sinh_hoc: double (nullable = true)
 |-- lich_su: double (nullable = true)
 |-- dia_li: double (nullable = true)
 |-- gdcd: double (nullable = true)

+--------+----+-------+---------+------+-------+--------+-------+------+----+
|     sbd|toan|ngu_van|ngoai_ngu|vat_li|hoa_hoc|sinh_hoc|lich_su|dia_li|gdcd|
+--------+----+-------+---------+------+-------+--------+-------+------+----+
|01000001| 3.6|    5.0|      4.0|  null|   null|    null|   2.75|   6.0|8.75|
|01000002| 8.4|   6.75|      7.6|  null|   null|    null|    8.5|   7.5|8.25|
|01000003| 5.8|    7.5|      5.0|  null|   null|    null|   7.25|   5.5|8.75|
|01000004| 7.4|    7.5|      8.6|  null|   null|    null|    7.5|   6.5| 7.5|
|01000005| 7.2|    8.5|      9.0|  null|   null|    null|    8.0|  

In [6]:
data_frame.createOrReplaceTempView("student")

In [7]:
def count_thi_xa_hoi():
    sql = spark.sql("select count(*) from student where vat_li is null and hoa_hoc is null and sinh_hoc is null")
    sql.show()

In [8]:
count_thi_xa_hoi()

+--------+
|count(1)|
+--------+
|  666206|
+--------+

